In [12]:
import pandas as pd
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from typing import List, Tuple
from collections import Counter
from scipy.stats import entropy as shannon_entropy

In [3]:
PARTY_SIZE = 4
SAMPLE_SIZE = 1000

In [ ]:
# 1. Data Loader
def prepare_data(qrng_path: str, threat_path: str):
    """Load angle and threat bit data from CSVs and return as numpy arrays."""
    qrng = pd.read_csv(qrng_path)
    threat = pd.read_csv(threat_path)

    assert qrng.shape == threat.shape == (SAMPLE_SIZE, PARTY_SIZE), "Shape mismatch or incorrect constants."

    return qrng.values, threat.values.astype(int)

In [ ]:
# 2. Circuit Builder
def build_qsmpc_circuit(angles: List[float], bits: List[int]) -> QuantumCircuit:
    """Build a Q-SMPC circuit for given angles and input bits."""
    qc = QuantumCircuit(PARTY_SIZE, PARTY_SIZE)

    # Graph state: H + CZ chain
    qc.h(range(PARTY_SIZE))
    for i in range(PARTY_SIZE - 1):
        qc.cz(i, i + 1)

    # Private angle rotations
    for i in range(PARTY_SIZE):
        qc.rz(angles[i], i)

    # Conditional X-basis measurement
    for i in range(PARTY_SIZE):
        if bits[i] == 1:
            qc.h(i)
        qc.measure(i, i)

    return qc

In [ ]:
def majority_vote_bitstring(bitstrings: List[str]) -> str:
    """Given a list of bitstrings (e.g. ['0111', '0011']), return majority bitstring per qubit."""
    
    length = len(bitstrings[0])
    vote_result = ''

    for i in range(length):
        ith_bits = [bits[i] for bits in bitstrings]
        common_bit = Counter(ith_bits).most_common(1)[0][0]
        vote_result += common_bit

    return vote_result

In [ ]:
def xor_bitstring(bitstring: str) -> int:
    """XOR all bits in a bitstring and return a single bit result."""
    return int(np.bitwise_xor.reduce([int(b) for b in bitstring]))

In [ ]:
# 3. Round Runner
def simulate_round(angles: List[float], bits: List[int], backend, shots: int = 1) -> List[str]:
    """Run a single Q-SMPC round and return raw measurement bitstrings."""
    qc = build_qsmpc_circuit(angles, bits)
    compiled = transpile(qc, backend)
    result = backend.run(compiled, shots=shots).result()
    return list(result.get_counts().keys())

In [ ]:
def majority_classical(bits: List[int]) -> int:
    """Returns 1 if majority (3 or more) of bits are 1."""
    return int(sum(bits) >= 3)

In [ ]:
def or_classical(bits: List[int]) -> int:
    """Returns 1 if any party saw the threat (bit is 1)."""
    return int(any(bits))

In [ ]:
def compute_metrics(predictions: List[int], true_bits: np.ndarray, angles: np.ndarray) -> Tuple[float, float, float]:
    """
    predictions: list of 0/1 quantum output bits (length = SAMPLE_SIZE)
    true_bits: full threat_bits array of shape (SAMPLE_SIZE, PARTY_SIZE)
    angles: full qrng_angles array of shape (SAMPLE_SIZE, PARTY_SIZE)
    
    Returns:
        - Accuracy vs majority
        - Accuracy vs OR
        - Shannon entropy of angle distribution
    """
    assert len(predictions) == len(true_bits) == len(angles)

    correct_majority = 0
    correct_or = 0

    for i in range(len(predictions)):
        pred = predictions[i]
        true_maj = majority_classical(true_bits[i])
        true_or = or_classical(true_bits[i])

        if pred == true_maj:
            correct_majority += 1
        if pred == true_or:
            correct_or += 1

    acc_majority = correct_majority / len(predictions)
    acc_or = correct_or / len(predictions)

    # QRNG entropy over all angle values
    flat_angles = angles.flatten()
    hist, _ = np.histogram(flat_angles, bins=32, range=(0, 2 * np.pi), density=True)
    entropy_val = shannon_entropy(hist)

    return acc_majority, acc_or, entropy_val

In [ ]:
def run_all_samples(
    angles_data: np.ndarray,
    bits_data: np.ndarray,
    backend,
    shots_per_round: int = 16,
    verbose: bool = False
) -> List[int]:
    """
    Run the full Q-SMPC protocol for all samples.
    
    Returns a list of 0/1 quantum-derived output bits.
    """
    predictions = []

    for i in range(SAMPLE_SIZE):
        angles = angles_data[i]
        bits = bits_data[i]

        bitstrings = simulate_round(angles, bits, backend, shots=shots_per_round)
        majority = majority_vote_bitstring(bitstrings)
        output_bit = xor_bitstring(majority)

        predictions.append(output_bit)

        if verbose and (i < 5 or i % 100 == 0):  # limit printouts
            print(f"[{i+1}/{SAMPLE_SIZE}] output={output_bit}, bits={bits.tolist()}, angles={np.round(angles, 2)}")

    return predictions


In [ ]:
# Load data and backend
angles_data, bits_data = prepare_data('./data/qrng_angles.csv', './data/threat_bits.csv')
backend = AerSimulator()

# Run full batch
predictions = run_all_samples(angles_data, bits_data, backend, shots_per_round=16, verbose=True)

# Evaluate results
acc_maj, acc_or, entropy_val = compute_metrics(predictions, bits_data, angles_data)

print(f"\nAccuracy vs Majority: {acc_maj:.3f}")
print(f"Accuracy vs OR:       {acc_or:.3f}")
print(f"QRNG Entropy:         {entropy_val:.3f} bits")


[1/1000] output=1, bits=[0, 0, 1, 1], angles=[5.5  5.94 2.06 2.04]
[2/1000] output=0, bits=[1, 1, 1, 1], angles=[2.33 2.99 4.84 1.15]
[3/1000] output=1, bits=[0, 1, 0, 1], angles=[0.96 1.84 4.17 5.89]
[4/1000] output=0, bits=[0, 0, 1, 0], angles=[0.12 3.24 1.77 5.38]
[5/1000] output=1, bits=[1, 1, 0, 0], angles=[2.18 2.04 0.44 2.26]
[101/1000] output=1, bits=[0, 1, 0, 0], angles=[2.82 1.5  2.97 4.44]
[201/1000] output=1, bits=[1, 1, 0, 0], angles=[4.34 3.95 4.37 4.12]
[301/1000] output=1, bits=[1, 0, 0, 0], angles=[1.99 1.23 5.03 0.29]
[401/1000] output=1, bits=[1, 1, 0, 1], angles=[4.17 5.08 5.84 3.93]
[501/1000] output=0, bits=[1, 1, 0, 0], angles=[3.85 3.78 3.83 5.89]
[601/1000] output=1, bits=[0, 1, 0, 0], angles=[0.12 5.15 4.74 1.2 ]
[701/1000] output=0, bits=[1, 0, 1, 1], angles=[3.66 0.12 5.13 2.48]
[801/1000] output=0, bits=[0, 1, 0, 0], angles=[2.36 4.47 1.72 4.32]
[901/1000] output=0, bits=[1, 1, 1, 1], angles=[5.13 3.19 0.81 1.74]

✅ Accuracy vs Majority: 0.497
✅ Accuracy vs